In [ ]:
import zipfile, os, cv2
import numpy as np
import tensorflow as tf
tf.executing_eagerly()
from tensorflow import keras
# Display
from keras import backend as K
import pandas as pd

In [ ]:
size = 192
depth = 32
seed = 2
# seed: cv1=1 cv2=2 cv3=3
save_nii_path='../nii_save_np/all_seg_data/'
ALL_nii_path = sorted(os.listdir('../isXXXX_all_mask_and_image/'))
print(len(ALL_nii_path))
print(ALL_nii_path[0][6:-7])


In [ ]:
masks_path=[]
image_path=[]
for i in ALL_nii_path: #/ssd1/cnn/Classification/is0001-is0154_mask_and_image/is0090s.nii.gz
    if 'o' in i:
        if 'nii.gz' in i:
            image_path.append('../isXXXX_all_mask_and_image/'+i.replace('s.nii.gz','o.nii.gz'))
        else:
            image_path.append('../isXXXX_all_mask_and_image/'+i.replace('s.nii','o.nii'))
    else:
        masks_path.append('../isXXXX_all_mask_and_image/'+i)

print(len(image_path), len(masks_path))

In [ ]:
# print(np.array(image_path[170:-1]))
# print(np.array(masks_path[170:-1]))

In [ ]:
import nibabel as nib
from skimage import morphology
from scipy import ndimage
from PIL import Image
import numpy as np
def normalize(volume, norm_type):
    if norm_type == 'zero_mean':
        img_o = np.float32(volume.copy())
        m = np.mean(img_o)
        s = np.std(img_o)
        volume = np.divide((img_o - m), s)
    elif norm_type == 'div_by_max':
        volume = np.divide(volume, np.percentile(volume,98))
        
    elif norm_type == 'onezero':
        for channel in range(volume.shape[-1]):
            volume_temp = volume[..., channel]
            volume_temp = (volume_temp - np.min(volume_temp)) / (np.max(volume_temp)-np.min(volume_temp))

            volume[..., channel] = volume_temp
    volume = volume.astype("float32")
    return volume

def resize_volume(img,size,depth):
    """Resize across z-axis"""
    # Set the desired depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
        # Rotate img shape = (height, wight, depth)
    for i in range(img.shape[2]):
        img[:,:,i] = np.fliplr(np.flipud(img[:,:,i]))
#     img = ndimage.rotate(img, 180, reshape=False, mode="nearest")
    img = ndimage.zoom(img, (size/current_height, size/current_width, 1), order=0)
    return img

def process_scan(path):
# get nib first channel
    try:
        image = nib.load(path)
        if len(image.shape) == 4:
            image = image.get_fdata()
            width,height,queue,_ = image.shape
            image = image[:,:,:,1]
            image = np.reshape(image,(width,height,queue))
        else:
            image = image.get_fdata()
            pass
        volume = normalize(image,"onezero")
        volume = resize_volume(volume,size,depth)
    #   add only black background mri image
        if volume.shape[2]!=depth:
            add_black_num = depth - volume.shape[2]
            volume = np.transpose(volume)
            for i in range(add_black_num):
                add_black_ = np.expand_dims(np.zeros((volume.shape[2],volume.shape[2])),axis=0)
                volume = np.concatenate((volume, add_black_), axis = 0)
            volume = np.transpose(volume)
        volume = np.transpose(volume)
    except:
        pass
        print(path)
        volume = image = np.zeros((192,192,24))
    return volume
def mask_scan(path):
# get nib first channel
    try:
        image = nib.load(path)
        if len(image.shape) == 4:
            image = image.get_fdata()
            width,height,queue,_ = image.shape
            image = image[:,:,:,1]
            image = np.reshape(image,(width,height,queue))
        else:
            image = image.get_fdata()
            pass

        image = resize_volume(image,size,depth)
        shape = image.shape
    #   add only black background mri image
        if image.shape[2]!=depth:
            add_black_num = depth - image.shape[2]
            image = np.transpose(image)
            for i in range(add_black_num):
                add_black_ = np.expand_dims(np.zeros((image.shape[2],image.shape[2])),axis=0)
                image = np.concatenate((image, add_black_), axis = 0)
            image = np.transpose(image)
        image = np.transpose(image)
    except:
        pass
        image = np.zeros((192,192,24))
        print(path)
    return image

In [ ]:
# nii load
from tqdm import tqdm
# from utils.nii_load import *
image_arr = np.array([process_scan(path) for path in tqdm(image_path)])
masks_arr = np.array([mask_scan(path) for path in tqdm(masks_path)])
# image_arr = np.array([process_scan(path) for path in image_path])
# masks_arr = np.array([mask_scan(path) for path in masks_path])

In [ ]:
print(image_arr.shape, masks_arr.shape)
print(np.max(image_arr), np.min(image_arr))

In [ ]:
np.save(f'{save_nii_path}image_arr',image_arr)
np.save(f'{save_nii_path}masks_arr',masks_arr)

In [ ]:
# nii array processing
image_arr = np.load(f'{save_nii_path}image_arr.npy')
masks_arr = np.load(f'{save_nii_path}masks_arr.npy')
print(image_arr.shape, masks_arr.shape)

In [ ]:
# Classification extract image_seg area
try:
    image_mask_arr_t = np.zeros((image_arr_t.shape[0],32,192,192))
    image_mask_arr_f = np.zeros((image_arr_f.shape[0],32,192,192))

    def mask_stroke_area(image_, mask_):
        image_save = image_.copy()
        for i in range(image_.shape[0]):
            for j in range(image_.shape[1]):
                for k in range(image_.shape[2]):
                    if mask_[i][j][k]==0:
                        image_save[i][j][k]=0
        return image_save

    # for i in tqdm(range(cost)):
    for i in tqdm(range(image_arr_t.shape[0])):
        image_mask_arr_t[i] = mask_stroke_area(image_arr_t[i],masks_arr_t[i])

    for i in tqdm(range(image_arr_f.shape[0])):
        image_mask_arr_f[i] = mask_stroke_area(image_arr_f[i],masks_arr_f[i])
except:
    print('pass')

In [ ]:
# Classification extract image_seg area save
try:
    np.save(f'{save_nii_path}image_mask_arr_t',image_mask_arr_t)
    np.save(f'{save_nii_path}image_mask_arr_f',image_mask_arr_f)
    image_mask_arr_t = np.load(f'{save_nii_path}image_mask_arr_t.npy')
    image_mask_arr_f = np.load(f'{save_nii_path}image_mask_arr_f.npy')
except:
    print('pass')

In [ ]:
# Classification label array
try:
    image_label_t = np.array([1. for _ in range(image_arr_t.shape[0])])
    image_label_f = np.array([0. for _ in range(image_arr_f.shape[0])])
    print(image_label_t.shape, image_label_f.shape)
    print(image_label_t[0:5], image_label_f[0:5])
    np.save(f'{save_nii_path}image_label_t',image_label_t)
    np.save(f'{save_nii_path}image_label_f',image_label_f)
    image_label_t = np.load(f'{save_nii_path}image_label_t.npy')
    image_label_f = np.load(f'{save_nii_path}image_label_f.npy')
    print(image_label_t.shape, image_label_f.shape)
except:
    print('pass')

In [ ]:
# fold 3 shuffle data
from keras.utils.np_utils import to_categorical

# random shuffle dataset
def random_data_shuffle(data_arr):
    np.random.seed(seed)
    np.random.shuffle(data_arr) #image
    return data_arr
# segmentation npy data processing

try:
    image_arr = random_data_shuffle(image_arr)
    masks_arr = random_data_shuffle(masks_arr)
except:
    print('pass')
    
# Classification npy data processing
try:
    image_mask_arr_t = random_data_shuffle(image_mask_arr_t)
    image_label_t = random_data_shuffle(image_label_t)
    image_mask_arr_f = random_data_shuffle(image_mask_arr_f)
    image_label_f = random_data_shuffle(image_label_f)
except:
    print('pass')
    
# path
try:
    img_path = random_data_shuffle(image_path)
except:
    print('pass')

In [ ]:
# scale data size cv
_size = int(image_arr.shape[0]*0.8)
print(_size)

In [ ]:
# segmentation npy data processing ,cv
try:
    sx_train = image_arr[:_size]
    sy_train = masks_arr[:_size]
    sx_val = image_arr[_size:]
    sy_val = masks_arr[_size:]
    print(sx_train.shape, sy_train.shape, sx_val.shape, sy_val.shape)
except:
    print('pass')

In [ ]:
# Classification npy data processing ,cv
try:
    cx_train = np.concatenate((image_mask_arr_t[:t_size],image_mask_arr_f[:f_size]),axis=0)
    cy_train = np.concatenate((image_label_t[:t_size], image_label_f[:f_size]), axis=0)

    cx_val = np.concatenate((image_mask_arr_t[t_size:], image_mask_arr_f[f_size:]), axis=0)
    cy_val = np.concatenate((image_label_t[t_size:], image_label_f[f_size:]), axis=0)

    print(cx_train.shape, cy_train.shape, cx_val.shape, cy_val.shape)
except:
    print('pass')

In [ ]:
# Save data name and data path ,cv
try:
    train_path = img_path[:_size]
    valid_path = img_path[_size:]
    train_val_path = np.concatenate((train_path,valid_path))
    print(train_val_path[0])
except:
    print('pass')

In [ ]:
# Save segmentation npy data processing ,cv
try:
    strain_val_x = np.concatenate((sx_train,sx_val))
    strain_val_y = np.concatenate((sy_train,sy_val))
    print(strain_val_x.shape, strain_val_y.shape)
except:
    print('pass')

In [ ]:
# Save classification npy data processing ,cv
try:
    ctrain_val_x = np.concatenate((cx_train,cx_val))
    ctrain_val_y = np.concatenate((cy_train,cy_val))
    print(ctrain_val_x.shape, ctrain_val_y.shape)
except:
    print('pass')

In [ ]:
# Save data name and data path ,cv
cv = seed
try:
    np.save(save_nii_path+f'cv{cv}_path_train_val',train_val_path)
    print(train_val_path.shape)
except:
    print('pass')
# Save segmentation npy data processing ,cv
try:
    np.save(save_nii_path+f'cv{cv}_x_strain_val',strain_val_x)
    np.save(save_nii_path+f'cv{cv}_y_strain_val',strain_val_y)
    print(strain_val_x.shape,strain_val_y.shape)
except:
    print('pass')
# Save classification npy data processing ,cv
try:
    np.save(save_nii_path+f'cv{cv}_x_ctrain_val',ctrain_val_x)
    np.save(save_nii_path+f'cv{cv}_y_ctrain_val',ctrain_val_y)
    print(ctrain_val_x.shape,ctrain_val_y.shape)
except:
    print('pass')